In [1]:
import csv
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

Ucitavanje baze i njeno analiziranje:

In [2]:
data = pd.read_csv('diabetic_data.csv',na_values='*')
naziv_atributa = data.columns
for col in naziv_atributa:
    print(col)

encounter_id
patient_nbr
race
gender
age
weight
admission_type_id
discharge_disposition_id
admission_source_id
time_in_hospital
payer_code
medical_specialty
num_lab_procedures
num_procedures
num_medications
number_outpatient
number_emergency
number_inpatient
diag_1
diag_2
diag_3
number_diagnoses
max_glu_serum
A1Cresult
metformin
repaglinide
nateglinide
chlorpropamide
glimepiride
acetohexamide
glipizide
glyburide
tolbutamide
pioglitazone
rosiglitazone
acarbose
miglitol
troglitazone
tolazamide
examide
citoglipton
insulin
glyburide-metformin
glipizide-metformin
glimepiride-pioglitazone
metformin-rosiglitazone
metformin-pioglitazone
change
diabetesMed
readmitted


In [3]:
data.head()
data.shape


(101766, 50)

In [4]:
#readmitted(ponovni prijem) - indikator da li je pacijent ponovo 
#primljen u bolnicu u roku od 30 dana nakon prvog prijema

#gledamo balansiranost podataka
broj_klasa = data['readmitted'].value_counts()
broj_klasa

NO     54864
>30    35545
<30    11357
Name: readmitted, dtype: int64

In [5]:
#sledeci deo radim da bih na jednom mestu imala koji su to sve
#kategoricki a koji su numericki atributi

numerical_cols = data.select_dtypes(include=['int64', 'float64']).columns
print('Numericki podaci su:\n')
for num in numerical_cols:
    print(num)
    
print('\n')
    
categorical_cols = data.select_dtypes(include=['object', 'category']).columns
print('Kategoricki podaci su:\n')
for cat in categorical_cols:
    print(cat)

Numericki podaci su:

encounter_id
patient_nbr
admission_type_id
discharge_disposition_id
admission_source_id
time_in_hospital
num_lab_procedures
num_procedures
num_medications
number_outpatient
number_emergency
number_inpatient
number_diagnoses


Kategoricki podaci su:

race
gender
age
weight
payer_code
medical_specialty
diag_1
diag_2
diag_3
max_glu_serum
A1Cresult
metformin
repaglinide
nateglinide
chlorpropamide
glimepiride
acetohexamide
glipizide
glyburide
tolbutamide
pioglitazone
rosiglitazone
acarbose
miglitol
troglitazone
tolazamide
examide
citoglipton
insulin
glyburide-metformin
glipizide-metformin
glimepiride-pioglitazone
metformin-rosiglitazone
metformin-pioglitazone
change
diabetesMed
readmitted


In [6]:
nedostajuce = data.isna().all().sum() 
print('\nProvera, nedostajucih ima:')
nedostajuce


Provera, nedostajucih ima:


0

In [7]:
for x in categorical_cols:
    print(data[x].unique())

['Caucasian' 'AfricanAmerican' '?' 'Other' 'Asian' 'Hispanic']
['Female' 'Male' 'Unknown/Invalid']
['[0-10)' '[10-20)' '[20-30)' '[30-40)' '[40-50)' '[50-60)' '[60-70)'
 '[70-80)' '[80-90)' '[90-100)']
['?' '[75-100)' '[50-75)' '[0-25)' '[100-125)' '[25-50)' '[125-150)'
 '[175-200)' '[150-175)' '>200']
['?' 'MC' 'MD' 'HM' 'UN' 'BC' 'SP' 'CP' 'SI' 'DM' 'CM' 'CH' 'PO' 'WC' 'OT'
 'OG' 'MP' 'FR']
['Pediatrics-Endocrinology' '?' 'InternalMedicine'
 'Family/GeneralPractice' 'Cardiology' 'Surgery-General' 'Orthopedics'
 'Gastroenterology' 'Surgery-Cardiovascular/Thoracic' 'Nephrology'
 'Orthopedics-Reconstructive' 'Psychiatry' 'Emergency/Trauma'
 'Pulmonology' 'Surgery-Neuro' 'Obsterics&Gynecology-GynecologicOnco'
 'ObstetricsandGynecology' 'Pediatrics' 'Hematology/Oncology'
 'Otolaryngology' 'Surgery-Colon&Rectal' 'Pediatrics-CriticalCare'
 'Endocrinology' 'Urology' 'Psychiatry-Child/Adolescent'
 'Pediatrics-Pulmonology' 'Neurology' 'Anesthesiology-Pediatric'
 'Radiology' 'Pediatrics-Hematol

['No']
['No']
['No' 'Up' 'Steady' 'Down']
['No' 'Steady' 'Down' 'Up']
['No' 'Steady']
['No' 'Steady']
['No' 'Steady']
['No' 'Steady']
['No' 'Ch']
['No' 'Yes']
['NO' '>30' '<30']


In [8]:
upitnik = (data == '?').any(axis=0)
print(upitnik)
print('\n')
none_vrednosti=(data =='None').any(axis=0)
print(none_vrednosti)

encounter_id                False
patient_nbr                 False
race                         True
gender                      False
age                         False
weight                       True
admission_type_id           False
discharge_disposition_id    False
admission_source_id         False
time_in_hospital            False
payer_code                   True
medical_specialty            True
num_lab_procedures          False
num_procedures              False
num_medications             False
number_outpatient           False
number_emergency            False
number_inpatient            False
diag_1                       True
diag_2                       True
diag_3                       True
number_diagnoses            False
max_glu_serum               False
A1Cresult                   False
metformin                   False
repaglinide                 False
nateglinide                 False
chlorpropamide              False
glimepiride                 False
acetohexamide 

Po ovome smo dobili da su atributi:race, weight, payer_code, medical_specialty, 
diag_1, diag_2 ,diag_3, max_glu_serum, A1Cresult .... oni atributi koji koji imaju nedostajuce vrednosti.
Međutim, neki od ovih atributa mogu biti manje relevantni za ciljnu varijablu "readmitted", kao što su race, 
weight i payer_code(kod osiguravajuće kompanije), pa bi se mogli izostaviti iz skupa atributa. 
S druge strane, atributi diag_1(primarna dijagnoza), diag_2(sekundarna dijagnoza), diag_3(tercijarna dijagnoza),
max_glu_serum i A1Cresult mogu biti važni jer se tiču dijagnostike i tretmana dijabetesa,
što može biti povezano sa ponovnim prijemom u bolnicu.


In [9]:
# Popunjavanje nedostajućih vrednosti sa najčešćom vrednošću u koloni(jer su ovi "bitni" atributi relevantsni
#i vazni za nasu bazu i za razmatranje ponovnog prijema u bolnici)

bitni=['diag_1', 'diag_2', 'diag_3', 'max_glu_serum' ,'A1Cresult']


for b in bitni:
    most_frequent_value = data[b].mode().values[0]
    data[b] = data[b].replace(['?', 'None'], most_frequent_value)

In [10]:
for x in categorical_cols:
    print(data[x].unique())

['Caucasian' 'AfricanAmerican' '?' 'Other' 'Asian' 'Hispanic']
['Female' 'Male' 'Unknown/Invalid']
['[0-10)' '[10-20)' '[20-30)' '[30-40)' '[40-50)' '[50-60)' '[60-70)'
 '[70-80)' '[80-90)' '[90-100)']
['?' '[75-100)' '[50-75)' '[0-25)' '[100-125)' '[25-50)' '[125-150)'
 '[175-200)' '[150-175)' '>200']
['?' 'MC' 'MD' 'HM' 'UN' 'BC' 'SP' 'CP' 'SI' 'DM' 'CM' 'CH' 'PO' 'WC' 'OT'
 'OG' 'MP' 'FR']
['Pediatrics-Endocrinology' '?' 'InternalMedicine'
 'Family/GeneralPractice' 'Cardiology' 'Surgery-General' 'Orthopedics'
 'Gastroenterology' 'Surgery-Cardiovascular/Thoracic' 'Nephrology'
 'Orthopedics-Reconstructive' 'Psychiatry' 'Emergency/Trauma'
 'Pulmonology' 'Surgery-Neuro' 'Obsterics&Gynecology-GynecologicOnco'
 'ObstetricsandGynecology' 'Pediatrics' 'Hematology/Oncology'
 'Otolaryngology' 'Surgery-Colon&Rectal' 'Pediatrics-CriticalCare'
 'Endocrinology' 'Urology' 'Psychiatry-Child/Adolescent'
 'Pediatrics-Pulmonology' 'Neurology' 'Anesthesiology-Pediatric'
 'Radiology' 'Pediatrics-Hematol

['No' 'Steady']
['No' 'Steady' 'Up' 'Down']
['No' 'Steady' 'Up' 'Down']
['No' 'Steady']
['No' 'Steady' 'Up' 'Down']
['No' 'Steady' 'Up' 'Down']
['No' 'Steady' 'Up' 'Down']
['No' 'Steady' 'Down' 'Up']
['No' 'Steady']
['No' 'Steady' 'Up']
['No']
['No']
['No' 'Up' 'Steady' 'Down']
['No' 'Steady' 'Down' 'Up']
['No' 'Steady']
['No' 'Steady']
['No' 'Steady']
['No' 'Steady']
['No' 'Ch']
['No' 'Yes']
['NO' '>30' '<30']


In [11]:
#zbog ponovne pojave "?" sam posumnjala da je mozda to i najfrekventnija vrednosta pa to i proveravamo:
mapa={}

ponovljeni=['weight','payer_code','medical_specialty']

for x in data['weight']:
    if x not in mapa.keys():
        mapa[x]=1
    else:
        mapa[x]=mapa[x]+1
        
mapa

{'?': 98569,
 '[75-100)': 1336,
 '[50-75)': 897,
 '[0-25)': 48,
 '[100-125)': 625,
 '[25-50)': 97,
 '[125-150)': 145,
 '[175-200)': 11,
 '[150-175)': 35,
 '>200': 3}

In [12]:
#Posto po ovome vidimo da nedostajucih vrednosti ima najvise(takodje i u payer_code i medicsl_specialty)
#sto nam dozvoljava da (naivno) obrisemo ceo red

In [13]:
data.drop(['weight','payer_code','medical_specialty','encounter_id','patient_nbr'], axis=1,inplace=True)

In [14]:
categorical_cols2 = data.select_dtypes(include=['object', 'category']).columns
print('Kategoricki podaci su:\n')
for cat in categorical_cols2:
    print(cat)

Kategoricki podaci su:

race
gender
age
diag_1
diag_2
diag_3
max_glu_serum
A1Cresult
metformin
repaglinide
nateglinide
chlorpropamide
glimepiride
acetohexamide
glipizide
glyburide
tolbutamide
pioglitazone
rosiglitazone
acarbose
miglitol
troglitazone
tolazamide
examide
citoglipton
insulin
glyburide-metformin
glipizide-metformin
glimepiride-pioglitazone
metformin-rosiglitazone
metformin-pioglitazone
change
diabetesMed
readmitted


In [15]:
# Primena one-hot encodinga na kategoričke atribute
#data_encoded=pd.get_dummies(data, columns=categorical_cols2)

# Prikaz prvih pet redova kodiranih podataka
#print(data_encoded)

In [16]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

for c in categorical_cols2:
    data[c] = label_encoder.fit_transform(data[c])


In [17]:
data.describe()

,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
count,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,...,101766.0,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000,101766.000000
mean,2.598776,0.462443,6.096702,2.024006,3.715642,5.754437,4.395987,43.095641,1.339730,16.021844,...,0.0,1.405469,1.006898,0.000128,0.000010,0.000020,0.000010,0.538048,0.770031,1.427520
std,0.938415,0.498649,1.594084,1.445403,5.280166,4.064081,2.985108,19.674362,1.705807,8.127566,...,0.0,0.838798,0.084414,0.011302,0.003135,0.004433,0.003135,0.498553,0.420815,0.684068
min,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,0.000000,5.000000,1.000000,1.000000,1.000000,2.000000,31.000000,0.000000,10.000000,...,0.0,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
50%,3.000000,0.000000,6.000000,1.000000,1.000000,7.000000,4.000000,44.000000,1.000000,15.000000,...,0.0,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,2.000000
75%,3.000000,1.000000,7.000000,3.000000,4.000000,7.000000,6.000000,57.000000,2.000000,20.000000,...,0.0,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,2.000000
max,5.000000,2.000000,9.000000,8.000000,28.000000,25.000000,14.000000,132.000000,6.000000,81.000000,...,0.0,3.000000,3.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000


In [18]:
X = data.drop(['readmitted'], axis=1)
y = data['readmitted']
print(X.shape,y.shape)

(101766, 44) (101766,)


Pripema podataka za klasifikaciju - podela na trening i test skupove:

In [45]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer, StandardScaler
from sklearn.linear_model import LogisticRegression


In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,)

In [21]:
normalizer = Normalizer()
scaler = StandardScaler()

In [37]:
X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)

In [38]:
X_train_normalized

array([[ 0.4256623 , -0.92800111,  0.56705313, ..., -0.00350476,
         0.92770312, -1.82526095],
       [ 0.4256623 ,  1.077372  , -0.68616862, ..., -0.00350476,
         0.92770312, -1.82526095],
       [ 0.4256623 ,  1.077372  ,  1.19366401, ..., -0.00350476,
         0.92770312,  0.54786687],
       ...,
       [ 0.4256623 ,  1.077372  ,  0.56705313, ..., -0.00350476,
        -1.07793106,  0.54786687],
       [ 0.4256623 , -0.92800111, -0.05955774, ..., -0.00350476,
        -1.07793106,  0.54786687],
       [ 0.4256623 , -0.92800111, -1.31277949, ..., -0.00350476,
        -1.07793106,  0.54786687]])

In [54]:
model = LogisticRegression(multi_class='multinomial', solver='saga', max_iter=1000)

In [55]:
model.fit(X_train_normalized,y_train)

LogisticRegression(max_iter=1000, multi_class='multinomial', solver='saga')

In [56]:
y_pred = model.predict(X_test_normalized)

In [57]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [58]:
confusion_matrix(y_test,y_pred)

array([[  21,  638, 1626],
       [  21, 1723, 5373],
       [  13, 1096, 9843]])

In [59]:
accuracy_score(y_test, y_pred)

0.5692738528053454

In [30]:
#plt.figure(figsize=(20,15))
#tree_plot = plot_tree(model)